In [ ]:
import numpy as np
import scipy as sp
import scipy.spatial.transform._rotation as R

This is a notebook to confirm the behaviour of scipy's Rotation library with respect to the aerospace conventions.

First thing is that the order appears to be roll,pitch,yaw. Notice that you can tell which is which by how the fix the corresponding vectors. This can be summarized as follows

|Rotation Name | Fixed Axis |
|--------------|------------|
|     Roll     |     x      |
|     Pitch    |     Y      |
|     Yaw      |     Z      |

Note that as you change the seq, it changes the matrix. Note however that "extrinsic" vs "intrinsic" does not matter. This leads to the first insight: 

*The sequence controls the meaning of the angles, not how they are multiplied*

From now on, we will assume the sequence is fixed at "XYZ", but this can be changed upon permuting the inputs. 

You may ask how the multiplication is ordered. We will come back to this in a minute. 

In [ ]:
seq = "ZYX"
R1 = R.Rotation.from_euler(angles=[np.pi/4,0,0],seq=seq)
R2 = R.Rotation.from_euler(angles=[0,np.pi/4,0],seq=seq)
R3 = R.Rotation.from_euler(angles=[0,0,np.pi/4],seq=seq)
print("First Angle")
print(np.round(R1.as_matrix(),2))
print("------------------------")
print(f"Second Angle")
print(np.round(R2.as_matrix(),2))
print("-----------------------")
print(f"Third Angle ")
print(np.round(R3.as_matrix(),2))

One thing we should note is that these are not as we expect. For instance according to the standard convention (passive rotations), $\psi = \pi/2$ should correspond to $$\begin{bmatrix} \cos(\psi) & sin(\psi) & 0 \\ -\sin(\psi) &\cos(\psi) & 0 \\ 0& 0 & 1\end{bmatrix} =\begin{bmatrix} 0 & 1 & 0 \\ -1 &0 & 0 \\ 0& 0 & 1\end{bmatrix}$$ 

In [ ]:
R3 = R.Rotation.from_euler(angles=[0,0,np.pi/2],seq="XYZ")
print(np.round(R3.as_matrix(),2))

We now come back to the notion of order. We intend for the 3-2-1 sequence which is given by the Yaw followed by Pitch followed by Roll. 

We have two options: "extrinsic" or "intrinsic". 

Let's determine the effect of this behaviour. We will do this via an experiment. We will compute a $\pi/2$ yaw rotation $R_3$ and a $\pi/2$ roll rotation $R_1$. We will then compare there combination to $R_1 R_3$ which is the correct ordering under three sequences: "XYZ","ZYX" (both are $\pi/2$), and "xyz". 

In [ ]:
R1 = R.Rotation.from_euler(angles=[np.pi/2,0,0],seq="XYZ").as_matrix()
R3 = R.Rotation.from_euler(angles=[0,0,np.pi/2],seq="XYZ").as_matrix()
comp_ext = R.Rotation.from_euler(angles=[np.pi/2,0,np.pi/2],seq="xyz").as_matrix()
comp_int = R.Rotation.from_euler(angles=[np.pi/2,0,np.pi/2],seq="XYZ").as_matrix()
comp_seq = R.Rotation.from_euler(angles=[np.pi/2,0,np.pi/2],seq="ZYX").as_matrix()
print(f"Extrinsic? {np.allclose(R1@R3,comp_ext)}")
print(f"Intrinsic? {np.allclose(R1@R3,comp_int)}")
print(f"Sequence matters? {not(np.allclose(comp_seq,comp_int))}")
print(f"Extrinsic is flipped? {np.allclose(R3@R1,comp_ext)}")
print(f"Sequence is flipped? {np.allclose(R3@R1,comp_seq)}")

This leads to a weird conclusion: 

 *The sequence specifies what axis each angles correspond to, but the input array controls the multiplication sequence. In particular the last in the array is always the first used for intrinsic and the opposite is true for extrinsic*

This leads us to thea few procedures:
- Input the angles as yaw,pitch,roll and use the sequence "ZYX" and then transpose
- Input the angles as roll,pitch,yaw and use the sequence "xyz" and then transpose
- Negate the angles, input as yaw,pitch,roll then use the sequence "zyx"
- Negate the angles, input as roll,pitch,yaw then use the sequence "XYZ"

We can quickly confirm that these procedures all give the same results with random angles

In [ ]:
def procedure_1(roll,pitch,yaw):
    Rot = R.Rotation.from_euler(angles=[yaw,pitch,roll],seq="ZYX").as_matrix().T
    return Rot

def procedure_2(roll,pitch,yaw):
    Rot = R.Rotation.from_euler(angles=[roll,pitch,yaw],seq="xyz").as_matrix().T
    return Rot

def procedure_3(roll,pitch,yaw):
    Rot = R.Rotation.from_euler(angles=[-yaw,-pitch,-roll],seq="zyx").as_matrix()
    return Rot

def procedure_4(roll,pitch,yaw):
    Rot = R.Rotation.from_euler(angles=[-roll,-pitch,-yaw],seq="XYZ").as_matrix()
    return Rot

In [ ]:
ran_roll = np.random.rand()*2*np.pi-np.pi
ran_pitch = np.random.rand()*np.pi-np.pi/2
ran_yaw = np.random.rand()*2*np.pi-np.pi

one_agree_two = np.allclose(procedure_1(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw),procedure_2(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
one_agree_three = np.allclose(procedure_1(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw),procedure_3(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
one_agree_four = np.allclose(procedure_1(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw),procedure_4(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
print(f"Procedure agree? {np.all([one_agree_two,one_agree_three,one_agree_four])}")

Okay, so these are consistent, but we need to make sure they agree with the general formula.

In [ ]:
def Rot_yaw(Psi):
    A = np.zeros((3,3))
    A[0,0] = np.cos(Psi)
    A[0,1] = np.sin(Psi)
    A[1,0] = -np.sin(Psi)
    A[1,1] = np.cos(Psi)
    A[2,2] = 1
    return A

def Rot_pitch(Theta):
    A = np.zeros((3,3))
    A[0,0] = np.cos(Theta)
    A[0,2] = -np.sin(Theta)
    A[2,0] = np.sin(Theta)
    A[2,2] = np.cos(Theta)
    A[1,1] = 1
    return A

def Rot_roll(Phi):
    A = np.zeros((3,3))
    A[1,1] = np.cos(Phi)
    A[1,2] = np.sin(Phi)
    A[2,1] = -np.sin(Phi)
    A[2,2] = np.cos(Phi)
    A[0,0] = 1
    return A

In [ ]:
real = Rot_roll(Phi=ran_roll)@Rot_pitch(Theta=ran_pitch)@Rot_yaw(Psi=ran_yaw)

print(f"Consistent? {np.allclose(real,procedure_1(ran_roll,ran_pitch,ran_yaw))}")

Now we need to check we can invert such a procedure. We provide inverse procedures for each of our earlier procedures.

In [ ]:
def inv_procedure_1(M):
    [yaw,pitch,roll] = R.Rotation.from_matrix(M.T).as_euler(seq="ZYX")
    return roll,pitch,yaw

def inv_procedure_2(M):
    [roll,pitch,yaw] = R.Rotation.from_matrix(M.T).as_euler(seq="xyz")
    return roll,pitch,yaw

def inv_procedure_3(M):
    [yaw,pitch,roll] = R.Rotation.from_matrix(M).as_euler(seq="zyx")
    return -roll, -pitch, -yaw

def inv_procedure_4(M):
    [roll,pitch,yaw]= R.Rotation.from_matrix(M).as_euler(seq="XYZ")
    return -roll,-pitch,-yaw

In [ ]:
ran_roll = np.random.rand()*2*np.pi-np.pi
ran_pitch = np.random.rand()*np.pi-np.pi/2
ran_yaw = np.random.rand()*2*np.pi-np.pi
out_roll,out_pitch,out_yaw = inv_procedure_1(procedure_1(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
print(f" Procedure 1 Works? {np.allclose([ran_roll,ran_pitch,ran_yaw],[out_roll,out_pitch,out_yaw])}")
out_roll,out_pitch,out_yaw = inv_procedure_2(procedure_2(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
print(f" Procedure 2 Works? {np.allclose([ran_roll,ran_pitch,ran_yaw],[out_roll,out_pitch,out_yaw])}")
out_roll,out_pitch,out_yaw = inv_procedure_3(procedure_3(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
print(f" Procedure 3 Works? {np.allclose([ran_roll,ran_pitch,ran_yaw],[out_roll,out_pitch,out_yaw])}")
out_roll,out_pitch,out_yaw = inv_procedure_4(procedure_4(roll=ran_roll,pitch=ran_pitch,yaw=ran_yaw))
print(f" Procedure 4 Works? {np.allclose([ran_roll,ran_pitch,ran_yaw],[out_roll,out_pitch,out_yaw])}")

Now let us confirm our procedure with some random angles

In [ ]:
N = 10
ran_roll = np.random.rand(N)*2*np.pi-np.pi
ran_pitch = np.random.rand(N)*np.pi-np.pi/2
ran_yaw = np.random.rand(N)*2*np.pi-np.pi

data = np.vstack((ran_yaw,ran_pitch,ran_roll)).T
R_arr = R.Rotation.from_euler(angles=-data,seq="zyx").as_matrix()

In [ ]:
n = np.random.choice(10)
A = R_arr[n,:,:]
R_test = procedure_1(roll=ran_roll[n],pitch=ran_pitch[n],yaw=ran_yaw[n])
print(f"Procedure for arrays works with random test? {np.allclose(R_test,A)}")

We now want to explain the notion of "active" and "passive" rotations so that we may understand how vectors of interest should transform under these actions.
Notice that up until this point we have been talking about how to transform "frames". What we mean by this is the following. Observe the following chart

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.arrow(0,0,1,0,head_width=0.03,color="blue",linewidth=1.5,label="Original X")
plt.arrow(0,0,0,1,head_width=0.03,color="green",linewidth=1.5,label = "Original Y")
plt.arrow(0,0,np.cos(np.pi/4),np.sin(np.pi/4),head_width=0.03,color="red",linewidth=1.5,label="New X")
plt.arrow(0,0,-np.sin(np.pi/4),np.cos(np.pi/4),head_width=0.02,color="orange",linewidth=1.5, label = "New Y")
plt.legend()
plt.show()

Now, imagine how you would explain the new frame relative to your frame. The simplest way would be to write out the new axeses in your frame. This would give you the following matrix:
$$\begin{bmatrix} \cos(\pi/4) & -\sin(\pi/4) \\ \sin(\pi/4) & \cos(\pi/4) \end{bmatrix}$$
Notice something crucial: This is not how you would map a point in your space to an equivalent set of coordinates in the second space. In fact, to map the point that corresponds to $[1,0]$ in the original frame to the corresponding point in the new frame, you would actually apply the inverse matrix:
$$\begin{bmatrix} \cos(\pi/4) & \sin(\pi/4) \\ -\sin(\pi/4) & \cos(\pi/4) \end{bmatrix}$$
as the point is actually in the "negative" x coordinate.
This is the fundamental difference between active and passive transformations. The passive transformation explains how frames map to one another, while active transformations tell you how coordinates change given the underlying point remains the same.
We will summarize this as follows:
- If $A$ is passive tranformation from fixed to body and $v$ must stay the same point in space expressed in the fixed frame, then to transform $v$ from fixed to the new frame is given by $A^T v$
- Thus if the above hold and $v$ is a physcial vector in the body frame, to transform it into the fixed frame we apply $Av$.

This may seem backwards (it did to me), but the inuition is that we are used to thinking of active transforms, not passive ones. In active transforms, the point itself moves. In passive, the point never moves. It is simply expressed in a different frame

Now, let us turn to angular velocity. Suppose that a frame is moving with a given angular velocity in my body frame. In particular, that means that the new frame's x-axis $r_1(t)$ must have a given angular velocity expressed in the fixed frame $\omega_f$ as a point mass relative to my origin. This means it must satisfy 
$$\dot{r}_1(t) = \omega_f \times r_1(t) = \widehat{\omega_f}r_1(t).$$
Where by $\hat{\omega_f}$ we just mean the matrix such that $\widehat{\omega_f} v = \omega_f \times v$ for all $v\in \mathbb{R}^n$ (this can be readily expressed via the Levi-Cevita symbol). 
Thus, we may express the frame of the rotating body relative to our fixed frame as 
$$\dot{R} = \widehat{\omega_f}R.$$
We however were given $\omega_B$ or the angular velocity expressed in the body frame. But remember, the angular velcoity is a physically meaningful vector, and so to translate it into the body frame we have that 
$$\dot{R} = \widehat{R\omega_B}R.$$
We now claim that $\widehat{R\omega_B}R = R\widehat{\omega_B}$. This reduces to the claim $Rv\times Rw = R (v\times w) \text{ for all R }\in SO(3)$ in the following way. Note that $\widehat{v}$ is uniquely defined. Consider the action on any fixed vector $w$. Then we have 
$$\widehat{R \omega_B} Rw = R\omega_B \times Rw, R\widehat{\omega_B}w = R (\omega_B \times w).$$
Thus if the subclaim is true, the two matrices are the same as they act the same on any given vector. Notice that a uniqueness condition of the cross-product is given as the unique vector $c = v\times w$ such that 
- $\|c\| = \|v\|\|w\|\sin(\theta)$
- $\langle c, v\rangle = \langle c , w \rangle = 0$
- $\det \begin{bmatrix} | & | & | \\ v & w & c \\ | & | & | \end{bmatrix} = 1$
Then notice that as $R$ preserves inner-products, norms and determinants if $R\in SO(3)$, it is clear that $Rc = Rv\times Rw$.

Therefore, we have that 
$$\dot{R} = R \widehat{\omega_B}.$$

This implies that the solution is given by
$$R(t) = R(0)\exp\left(\int_{0}^{t} \widehat{\omega_B}(s)ds\right)$$
And an expansion can be given in terms of $$\left\{ R, R\widehat{\omega_B}, R\widehat{\omega_B}^2, R\widehat{\omega_B}^3,...  \right\}$$